Paper<br>
https://arxiv.org/abs/2304.09790<br>
<br>
GitHub<br>
https://github.com/MCG-NKU/AMT<br>
<br>
<a href="https://colab.research.google.com/github/kaz12tech/ai_demos/blob/master/AMT_demo.ipynb" target="_blank"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Setup environment

## git clone

In [ ]:
%cd /content

!git clone https://github.com/MCG-NKU/AMT.git

%cd /content/AMT
# Commits on Apr 26, 2023
!git checkout 70f988fbfc0d3d458beba1ee49caf876e57968fe

## install libraries

In [ ]:
%cd /content/AMT

!pip install torch==1.11.0+cu113 torchvision==0.12.0+cu113 --extra-index-url https://download.pytorch.org/whl/cu113
!pip install omegaconf Pillow==9.4.0 tqdm==4.64.1
!pip install gdown

# install for video
!pip install yt-dlp
!pip install imageio==2.4.1 moviepy==0.2.3.5

## import libraries

In [ ]:
from moviepy.video.fx.resize import resize
from moviepy.editor import VideoFileClip

from yt_dlp import YoutubeDL


# Setup pretrained model

In [ ]:
%cd /content/AMT

%mkdir -p pretrained

!gdown 1WmOKmQmd6pnLpID8EpUe-TddFpJuavrL -O pretrained/amt-s.pth
!gdown 1UyhYpAQLXMjFA55rlFZ0kdiSVTL7oU-z -O pretrained/amt-l.pth
!gdown 1yieLtKh4ei3gOrLN1LhKSP_9157Q-mtP -O pretrained/amt-g.pth

# Setup video

## download video

In [ ]:
video_url = 'https://www.youtube.com/watch?v=muczNvx9fgg' #@param {type:"string"}

#@markdown 動画の切り抜き範囲(秒)を指定してください。\
#@markdown 30秒以上の場合OOM発生の可能性が高いため注意
start_sec =  0#@param {type:"integer"}
end_sec =  2#@param {type:"integer"}

(start_pt, end_pt) = (start_sec, end_sec)

In [ ]:
download_resolution = 720
full_video_path = './input/full_video.mp4'
input_clip_path = './input/clip_video.mp4'

# 動画ダウンロード
ydl_opts = {'format': f'best[height<={download_resolution}]', 'overwrites': True, 'outtmpl': full_video_path}
with YoutubeDL(ydl_opts) as ydl:
    ydl.download([video_url])

# 指定区間切り抜き
with VideoFileClip(full_video_path) as video:
    subclip = video.subclip(start_pt, end_pt)
    subclip.write_videofile(input_clip_path)

## show video

In [ ]:
# 取得動画の表示
clip = VideoFileClip(input_clip_path)
clip = resize(clip, height=420)
clip.ipython_display()

## video to frames

In [ ]:
!mkdir -p input/frames

!ffmpeg -i input/clip_video.mp4 -vf fps=25 input/frames/%06d.png

In [ ]:
!rm -rf input/target
!mkdir -p input/target

# copy 2 image
!cp input/frames/000001.png input/target
!cp input/frames/000025.png input/target

# Video Frame Interpolation

In [ ]:
!rm -rf {outputs_dir}

!python demos/demo_2x.py \
  --config cfgs/AMT-G.yaml \
  --ckpt pretrained/amt-g.pth \
  --niters 6 \
  --input assets/quick_demo \
  --out_path results_origin \
  --frame_rate 25 \
  --save_images

In [ ]:
# 取得動画の表示
clip = VideoFileClip('results_origin/demo_0000.mp4')
clip = resize(clip, height=380)
clip.ipython_display()

## setup path

In [ ]:
!rm -rf {outputs_dir}

!python demos/demo_2x.py \
  --config cfgs/AMT-G.yaml \
  --ckpt pretrained/amt-g.pth \
  --niters 6 \
  --input input/target \
  --out_path results \
  --frame_rate 25 \
  --save_images

In [ ]:
# 取得動画の表示
clip = VideoFileClip('results/demo_0000.mp4')
clip = resize(clip, height=380)
clip.ipython_display()